In [13]:
import pandas as pd

df = pd.read_csv('train-data.dat', sep=',', header=None)
df.columns = ['sentence']
df['sentence'] = df['sentence'].str.replace('<.*?>', '', regex=True)  # deletes <int>
# print(df.head())
df = df[0:1000]
df.head()

vocab = pd.read_csv('vocabs.txt', sep=',', header=None)
vocab.columns = ['word', 'id']

sentences = pd.read_csv('train-data.dat', sep=',', header=None)
sentences.head()

com = []
for line in df.sentence:
    sen = ''
    for word in line.split():
      for id in vocab.id:
        if int(word) == int(id):
          sen = sen + ' ' + str(vocab.word[id]) 
    # print(sen)
    com.append(sen)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

tfidf_vectorizer = TfidfVectorizer(use_idf=True)
tfidf_vectors = tfidf_vectorizer.fit_transform(com)

tfidf = tfidf_vectors.todense()
# TFIDF of words not in the doc will be 0, so replace them with nan
tfidf[tfidf == 0] = np.nan
# Use nanmean of numpy which will ignore nan while calculating the mean
means = np.nanmean(tfidf, axis=0)
# convert it into a dictionary for later lookup
means = dict(zip(tfidf_vectorizer.get_feature_names(), means.tolist()[0]))

tfidf = tfidf_vectors.todense()
# Argsort the full TFIDF dense vector
ordered = np.argsort(tfidf*-1)
words = tfidf_vectorizer.get_feature_names()

# top_k = 5
# for i, doc in enumerate(com):
#     result = {}
#     # Pick top_k from each argsorted matrix for each doc
#     for t in range(top_k):
#         # Pick the top k word, find its average tfidf from the
#         # precomputed dictionary using nanmean and save it to later use
#         result[words[ordered[i,t]]] = means[words[ordered[i,t]]]
#     # print (result)

# means

# from collections import Counter 

# means = dict(Counter(means).most_common(1000))


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [15]:
li = [[]]

for key in means:
  li.append([key, means[key]])

li.pop(0)
li[:5]

df2 = pd.DataFrame(li)
# df2 = df2.transpose()
df2.head()

tfidf_sc = df2[1]
print(tfidf_sc)
tfidf_len = len(tfidf_sc)
print(tfidf_len)

0       0.180064
1       0.105520
2       0.098096
3       0.163385
4       0.138688
          ...   
6848    0.110620
6849    0.151121
6850    0.124428
6851    0.184492
6852    0.223934
Name: 1, Length: 6853, dtype: float64
6853


In [1]:
# !pip install deap

import random

import matplotlib.pyplot as plt
from deap import base
from deap import creator
from deap import tools

POP_SIZE = 20
P_CROSSOVER = 0.6
P_MUTATION = 0.01
MAX_GENERATIONS = 200
GENERATIONS_EXIT = 20  # state the number of generations in which maxFitness remains the same,  for stopping condition


creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 160 kB 4.8 MB/s 


In [27]:
def pop_generator(POP_SIZE):
  li = [[random.randint(0,1) for x in range(tfidf_len)] for y in range(POP_SIZE)] 


  for i in range(len(li)):
    ones = 0
    for j in li[i]:
      if j==1:
        ones+=1
    # if ones<1000:
    while ones<4:
      print(ones)
      print(li[i])
      li[i] = [random.randint(0,1) for x in range(10)]
      print(li[i])


  li_df = pd.DataFrame(li)
  li_df.head()

  return li

In [82]:
# tfidf_sc
# df2.columns=['word', 'importance']
# df2.head(5)

def evaluation_function(individual):  
  a = np.mean(individual)
  return a

In [31]:
def set_fitness(population):
    ... # set fitness of non-dominant individuals to 0
    return population

In [83]:
tbx = base.Toolbox()

INDIVIDUAL_SIZE = 20

tbx.register("attr_int", random.randint, 0, 1)
tbx.register("individual", tools.initRepeat, creator.Individual, tbx.attr_int, n=INDIVIDUAL_SIZE)
tbx.register("population", tools.initRepeat, list, tbx.individual)
tbx.register("evaluate", evaluation_function)
# tbx.register("mate", tools.cxOnePoint)
# tbx.register("mutate", tools.mutFlipBit, indpb=0.01)
# tbx.register("select", tools.selTournament, tournsize=5)

In [84]:
## create random population,
population = tbx.population(n=100)

## set fitness,
# set_fitness(population)

population[:5]

[[1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0],
 [1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1],
 [0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0],
 [1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1],
 [1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0]]

In [96]:
evaluation = tbx.evaluate(df2.importance)

evaluation

fitnessValues = list(map(tbx.evaluate, population))

fitnessValues[:5]

[0.55, 0.55, 0.35, 0.5, 0.6]

In [81]:
iteration = 1
while iteration < 100:
    
    current_population = list(map(tbx.clone, population))
    
    offspring = []
    for _ in range(10):
      i1, i2 = np.random.choice(range(len(population)), size=2, replace=False)   
      offspring1, offspring2 = tbx.mate(population[i1], population[i2])  
      offspring.append(tbx.mutate(offspring1)[0])
      offspring.append(tbx.mutate(offspring2)[0])  
    
    for child in offspring:
        current_population.append(child)    
    
    ## reset fitness,
    set_fitness(current_population)   
    ## tourn. select, 
    population[:] = tbx.select(current_population, len(population))
    
    iteration += 1

AttributeError: ignored